# Section 5: Generators & Comprehension Expressions
- [Section 5: Generators & Comprehension Expressions](#section-5-generators-comprehension-expressions)
    - [Introducing Generators](#introducing-generators)
            - [The `range` generator](#the-range-generator)
        - [Creating your own generator: generator comprehensions](#creating-your-own-generator-generator-comprehensions)
            - [Storing generators](#storing-generators)
            - [Consuming generators](#consuming-generators)
            - [Using generator comprehensions on the fly](#using-generator-comprehensions-on-the-fly)
            - [List comprehensions](#list-comprehensions)
            - [Chaining comprehensions](#chaining-comprehensions)
            - [Nesting comprehensions](#nesting-comprehensions)
- [Generators are über powerful](#generators-are-%C3%BCber-powerful)

## Introducing Generators
Now we introduce an important type of iterable object called a **generator**, which allows use to generate arbitrarily-many items in a sequence of data, without having to store them all in memory at once.  

<div class="alert alert-block alert-info"> 
**Definition**: An **generator** is a special kind of iterable, which stores the instructions for how to *generate* each of its members, in order, along with its current state of iterations. It generates each member, one at a time, only as it is requested via iteration.
</div>

To contrast a generator with more familiar iterables, note that a list readily stores all of its members - you can access any of its contents via indexing. A generator, on the otherhand, *does not store any items*. Instead, it stores the instructions for generating each of its members in sequence, and stores its iteration state - meaning that the generator will know if it has generated its second member, and will thus generate its third member the next time it is iterated on. The whole point of this is that you can use a generator to produce a long sequence of items, without having to store them all in memory.

#### The `range` generator
The most popular built-in generator in Python is `range`, which, given the values: 
- 'start' (inclusive, default=0)
- 'stop' (exclusive)
- 'step' (default=1) 

will generate the corresponding sequence of integers upon iteration. Consider the following example usages of `range`:
```python
# start: 2  (included)
#  stop: 7  (excluded)
#  step: 1
for i in range(2, 7, 1):
    print(i)
    # prints: 2.. 3.. 4.. 5.. 6
```

```python
# start:  1  (included)
#  stop: 10  (excluded)
#  step:  2
for i in range(2, 7, 1):
    print(i)
    # prints: 1.. 3.. 5.. 7.. 9
```

```python
# A very common use case!
# start:  0  (default, included)
#  stop: 10  (excluded)
#  step:  1  (default)
for i in range(5):
    print(i)
    # prints: 0.. 1.. 2.. 3.. 4
```

Because `range` is a generator, the command `range(5)` will simply store the instructions needed to produce the sequence of numbers 0-4, whereas the list `[0, 1, 2, 3, 4]` stores all of these items in memory at once. For short sequences, this seems to be a rather paltry savings; this is not the case for long sequences. The following graph compares the memory consumption used when defining a generator for the sequence of number $0-N$ using `range`, compared to storing the sequence in a list:

![Mem_Consumption_Generator.png](./attachments/Mem_Consumption_Generator.png)

Given our discussion of generators, it should make sense that the memory consumed simply by defining `range(N)` is independent of $N$, whereas the memory consumed by the list grows linearly with $N$ (for large $N$).

<div class="alert alert-block alert-success"> 
**Takeaway**: `range` is a critically-important built-in generator, which generates sequences of integers.
</div>


### Creating your own generator: generator comprehensions
Python provides a sleek syntax for defining a simple generator in a single line of code; this expression is known as a **generator comprehension**. The following syntax is extremely useful and will appear very frequently in Python code:

<div class="alert alert-block alert-info"> 
**Definition**: The syntax 
<br>
`(<expression> for <var> in <iterable> [if <condition>])`
<br>
specifies the general form for a **generator comprehension**. This produces a generator, whose instructions for generating its members are provided within the parentheses of the statement. 
</div>

Written in a long form, the pseudo-code for `(<expression> for <var> in <iterable> if <condition>)` is:

```
(<expression> for <var> in <iterable> if <condition>), expressed in pseudo-code:

for <var> in <iterable>:
    if bool(<condition>):
        yield <expression>
```

The following expression defines a generator for all the even numbers in 0-99:
```python
# will generate 0.. 2.. 4.. ... 98
even_gen = (i for i in range(100) if i%2 == 0)
```
<div class="alert alert-block alert-warning"> 
**FYI**: A more efficient and concise way to generate this sequence of numbers is: `range(0, 100, 2)`
</div>


The `is <condition>` clause is in the generator expression is optional. The generator comprehension `(<expression> for <var> in <iterable>)` corresponds
to:

```
(<expression> for <var> in <iterable>), expressed in pseudo-code:

for <var> in <iterable>:
    yield <expression>
```

For example:
```python
# will generate 0/2.. 9/2.. 21/2.. 32/2
example_gen = (i/2 for i in [0, 9, 21, 32])
```

`<expression>` can be any valid single-line of Python code that returns an object:
```python
((i, i**2, i**3) for i in range(10))
# will generate:
# (0, 0, 0)
# (1, 1, 1)
# (2, 4, 8)
# (3, 9, 27)
# (4, 16, 64)
# (5, 25, 125)
# (6, 36, 216)
# (7, 49, 343)
# (8, 64, 512)
# (9, 81, 729)
```
<div class="alert alert-block alert-success"> 
**Takeaway**:  A generator comprehension is a single-line specification for defining a generator in Python. It is absolutely essential to learn this syntax in order to write simple and readable code.
</div>

<div class="alert alert-block alert-warning"> 
**Note**: Generator comprehensions are **not** the only method for defining generators in Python. We will briefly introduced a more fully-fledged syntax for defining generators, when we learn about `def` statements and functions.
</div>

#### Storing generators
Just like we saw with the `range` generator, defining a generator using a comprehension does *not* perform any computations or consumer any memory beyond defining the rules for producing the sequence of data. See what happens when we try to print this generator:
```python
# will generate 0, 1, 4, 9, 25, ..., 9801
>>> gen = (i**2 for i in range(100))
>>> print(gen)
<generator object <genexpr> at 0x000001E768FE8A40>
```
This output simply indicates that `gen` stores a generator-expression at the memory address `0x000001E768FE8A40`; this is simply where the instructions for generating our sequence of squared numbers is stored. `gen` will not produce any results until we iterate over it. For this reason, generators cannot be inspected in the same way that lists and other iterable containers can - you cannot query the length of a generator nor can you index into it.

<div class="alert alert-block alert-success"> 
**Takeaway**: Unlike objects like lists and tuples, a generator is merely a set of instructions, thus it cannot be printed and its individual items cannot accessed. (An exception to this is the built-in `range` generator, which can be indexed directly).
</div>

#### Consuming generators
We can feed this to any function that accepts iterables. For instance, we can feed `gen` to the built-in `sum` function, which sums the content of an iterable:
```python
# computes the sum 0 + 1 + 4 + 9 + 25 + ... + 9801
>>> gen = (i**2 for i in range(100))
>>> sum(gen)
328350
```
This computes the sum of the sequence of numbers *without ever storing the full sequence of numbers in memory*. In fact, only two numbers need be stored during any given iteration of the sum: the current value of the sum, and the number being added to it.

What happens if we run this command a second time:
```python
# computes the sum of ... nothing!
# `gen` has alredy been consumed!
>>> sum(gen)
0
```
It may be surprising to see that the sum now returns 0. This is because **a generator is exhausted after it is iterated over in full**. You must redefine the generator if you want to iterate over it again; fortunately, defining a generator requires very few resources, so this is not a point of concern.

<div class="alert alert-block alert-success"> 
**Takeaway**: A generator can only be iterated over once, after which it is exhausted and must be re-defined in order to be iterated over again.
</div>


#### Using generator comprehensions on the fly
A feature of Python that can make it supremely readable and intutive is that **generator comprehensions can be fed directly into functions**. That is,

```python
>>> gen = (i**2 for i in range(100))
>>> sum(gen)
328350
```

can be simplified as:

```python
>>> sum(i**2 for i in range(100))
328350
```

If I want my code to compute the finite harmonic series:
\begin{equation*}
\sum_{k=1}^{100} \frac{1}{n} = 1 + \frac{1}{2} + ... + \frac{1}{100} 
\end{equation*}

I can simply write:
```python
>>> sum(1/n for n in range(1, 101))
5.187377517639621
```

This convenient syntax works for function that expects an iterable as an argument, such as the `list` function:
```python
>>> list(i**2 for i in range(10))
[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
```
Using generator comprehensions to initialize lists is so useful, that Python actually reserves a specialized syntax for it, known as the **list comprehension**.

<div class="alert alert-block alert-success"> 
**Takeaway**: A generator comprehension can be specified directly as an argument to a function, wherever a single iterable is expected as an input to that function.
</div>

#### List comprehensions

<div class="alert alert-block alert-info"> 
**Definition**: A **list comprehension** is a syntax for initializing list, which exactly mirrors the generator comprehension syntax:
<br>
```
[<expression> for <var> in <iterable> {if <condition}]
```
</br>
</div>



For example, if we want to create a list of square-numbers, we can simply write:
```python
>>> [i**2 for i in range(10)]
[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
```

This produces the exact same result as feeding the `list` function a generator comprehension, however, **using a list comprehension is slightly more efficient than is feeding the `list` function a comprehension expression**:

In [13]:
%%timeit
list(i**2 for i in range(1000))

395 µs ± 15.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [14]:
%%timeit
[i**2 for i in range(1000)]

355 µs ± 14.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


If you want to intialize a list containing one hundred 0s (perhaps to be filled later with real data), you can simply write:
```python
[0 for i in range(100)]
```

<div class="alert alert-block alert-success"> 
**Takeaway**: A list comprehension is an extremely useful syntax for creating simple and complicated lists alike.
</div>

#### Chaining comprehensions
Because generators are iterables, they can be fed into subsequent generator comprehensions
```python
# generates 400.. 100.. 0.. 100.. 400 
>>> gen_1 = (i**2 for i in [-20, -10, 0, 10, 20])

# sums the generated numbers, excluding any numbers whose absolute value is greater than 150
>>> sum(i for i in gen_1 if abs(i) <= 150)
200
```

#### Nesting comprehensions
It can be useful to nest comprehensions within one another:
```python
# create a 3x4 "matrix" (list of lists) of zeros
>>> [[0 for col in range(4)] for row in range(3)]
[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]
```

<div class="alert alert-block alert-danger"> 
**Warning**: Chaining and nesting comprehension expressions should be used sparingly. Otherwise you can quickly end up with code that is nearly incomprehensible.
</div>

```python
# don't ever do this... ever
>>> [[(i if i>5 else i**2) for i in range(2*j -1) if i %2 ==0] for j in (i//2 for i in range(15))]
[[],
 [],
 [0],
 [0],
 [0, 4],
 [0, 4],
 [0, 4, 16],
 [0, 4, 16],
 [0, 4, 16, 6],
 [0, 4, 16, 6],
 [0, 4, 16, 6, 8],
 [0, 4, 16, 6, 8],
 [0, 4, 16, 6, 8, 10],
 [0, 4, 16, 6, 8, 10],
 [0, 4, 16, 6, 8, 10, 12]]
```

In [73]:
with open("all_of_wikipedia.txt", "w") as f:
    f.write(m)

# Generators are über powerful
How many times does the word "a" occur in wikipedia?
```python
with open("all_of_wikipedia.txt", mode="r") as f:
    sum(1 for word in
```

In [77]:
with open("all_of_wikipedia.txt", mode="r") as f:
    num_of_a = sum(sum(1 for word in sentence.split() if word.lower() == "a") for sentence in f)

In [78]:
num_of_a

4

In [48]:
m = "I am a dude I really am a dude a\n I am a dude"

In [70]:
lines = (line for line in m.splitlines())
words = (line.split() for line in lines)
sum(sum(1 for i in word if i == "a") for word in words)

4

In [64]:
list(words)

[['I', 'am', 'a', 'dude', 'I', 'really', 'am', 'a', 'dude', 'a'],
 ['I', 'am', 'a', 'dude']]